Vigenér cipher decryption
---

In [1]:
import os
import sys

__module_path = os.path.abspath(os.path.join('..'))
if __module_path not in sys.path:
    sys.path.append(__module_path)

import cryptoanalysis

import numpy as np

from IPython.display import Javascript

from plotly import graph_objs, __version__
import plotly.offline as plotly

from ipywidgets import interact, interactive, widgets, HBox

In [2]:
%matplotlib inline

In [3]:
plotly_version = __version__

# For online connection:
# plotly.tools.set_credentials_file(username='CermakM', api_key='q2yoGYw052dyHMd8ztqx')
# plotly.tools.set_config_file(world_readable=False, sharing='secret')

plotly.init_notebook_mode(connected=True)

Encrypted text

In [4]:
_submitted = False
_reset = True

cipher = ''
decrypted_text = ''

analyser = cryptoanalysis.decryption.Analyser(lang='en')

In [5]:
input_area = widgets.Textarea(
    placeholder='Paste here encoded text to be decrypted',
    description='Encrypted text:'
)

output_area = widgets.Textarea(
    placeholder='Submit cipher and start decrypting',
    description='Decrypted text:',
    disabled=True
)

In [6]:
submit_input_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',
    icon='check'
)

reset_input_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='danger'
)

In [7]:
def _on_submit(sender):
    global analyser, cipher, _submitted, _reset
            
    cipher = input_area.value
    
    if not cipher:
        return
    
    _reset = False

    analyser = cryptoanalysis.decryption.Analyser(cipher=cipher, lang='en')
    
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
    if not _submitted:
#         Javascript('$.notify("Cipher submitted");')
        print("Cipher submitted")
        
        _submitted = True
    
def _on_reset(sender):
    global analyser, cipher, _submitted
    
    _submitted = False
    
    cipher = ''
    analyser = cryptoanalysis.decryption.Analyser(lang='en')
    input_area.value = ''
    
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
    if not _reset:
#         Javascript('$.notify("Analyser has been reset")')
        print("Analyser has been reset")
        
        _reset = True

In [8]:
submit_input_button.on_click(_on_submit)
reset_input_button.on_click(_on_reset)

In [20]:
display(input_area)
HBox([submit_input_button, reset_input_button])

<IPython.core.display.Javascript object>

Cipher submitted


A Jupyter Widget

A Jupyter Widget

Frequency Analysis

In [21]:
default_lang_trace = graph_objs.Bar(
    x=analyser.alphabet,
    y=analyser.letter_frequency,
    name='Language'
)

if cipher:
    cipher_dict = analyser.get_char_frequency()
    cipher_trace = graph_objs.Bar(
        x=list(cipher_dict.keys()),
        y=list(cipher_dict.values()),
        name='Cipher'
    )
    data = [default_lang_trace, cipher_trace]

else:
    data = [default_lang_trace]
    
layout = graph_objs.Layout(
    title='Frequency analysis',
    xaxis=dict(tickangle=-45),
    yaxis=dict(tickformat=' %'),
    barmode='group'
)


In [22]:
fig = graph_objs.Figure(data=data, layout=layout)

plotly.iplot(fig)

Key length analysis

In [23]:
options = [] if not cipher else analyser.get_key_len_list()

widget_key_len = widgets.ToggleButtons(
    options=['-'] if not options else options,
    description='Suggested key lengths:',
    disabled=not options,
    button_style=''
)

In [24]:
display(widget_key_len)

A Jupyter Widget

Key decryption

In [25]:
key = ''
if _submitted:
    key, _ = analyser.get_keys(key_id=widget_key_len.value)

widget_key = widgets.Text(
    description='Decrypted key:',
    placeholder='Submit cipher and start decrypting',
    value=key,
    disabled=False
)

In [26]:
display(widget_key)

A Jupyter Widget

Text decryption

In [27]:
widget_rotation = widgets.ToggleButtons(
    options=[0, 1],
    description='Select rotation',
    disabled=False,
    tooltips=['Rotation 0: `a` -> `a`', 'Rotation 1: `a` -> `b`']
)

In [28]:
display(widget_rotation)

A Jupyter Widget

In [29]:
result = ''
if _submitted:
    result = analyser.decipher(custom_key=key, rot=widget_rotation.value)
output_area.value = result

In [30]:
display(output_area)

A Jupyter Widget